In [260]:
#Import the libraries
import math
# import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [261]:
import pyodbc
server = '' 
database = '' 
username = '' 
password = ''
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [272]:
query = """
SELECT 
PENGADAAN_ME5A_R1.BANFN,Pengadaan_ME5A_R1.BNFPO,
Pengadaan_ME2N.EBELN,Pengadaan_ME2N.EBELP, 
Pengadaan_ME2N.BEDAT, Pengadaan_ME2N.EKORG, Pengadaan_ME2N.EKGRP,
Pengadaan_ME2N.EMATN, Pengadaan_ME2N.TXZ01,
Pengadaan_ME2N.NETPR AS UNITPRICE, Pengadaan_ME2N.CURRENCY_RATE, Pengadaan_ME2N.WAERS,
Pengadaan_ME2N.MENGE AS QTY
FROM Pengadaan_ME2N
LEFT JOIN Pengadaan_ME5A_R1
ON Pengadaan_ME2N.BANFN = Pengadaan_ME5A_R1.BANFN AND Pengadaan_ME2N.BNFPO = Pengadaan_ME5A_R1.BNFPO
WHERE (Pengadaan_ME2N.BANFN IS NOT NULL) AND (Pengadaan_ME5A_R1.BADAT IS NOT NULL) AND
(Pengadaan_ME2N.EMATN = '000000000001000017'OR Pengadaan_ME2N.EMATN = '000000000003000499' 
OR Pengadaan_ME2N.EMATN = '000000000003000266' OR Pengadaan_ME2N.EMATN = '000000000003000508'
OR Pengadaan_ME2N.EMATN = '000000000003000539'
OR Pengadaan_ME2N.EMATN = '000000000003000018'OR Pengadaan_ME2N.EMATN = '000000000003000072' 
OR Pengadaan_ME2N.EMATN = '000000000003000048' OR Pengadaan_ME2N.EMATN = '000000000003000019')

ORDER BY PENGADAAN_ME5A_R1.BANFN,Pengadaan_ME5A_R1.BNFPO,Pengadaan_ME2N.EBELN
"""

df_all = pd.read_sql_query(query, cnxn)

In [273]:
df_curr = pd.read_csv('Data Historis USD_IDR.csv')

In [274]:
df_all['BEDAT'] = df_all['BEDAT'].str[6:8] + "/" + df_all['BEDAT'].str[4:6] + "/" + df_all['BEDAT'].str[:4]
df_all

,BANFN,BNFPO,EBELN,EBELP,BEDAT,EKORG,EKGRP,EMATN,TXZ01,UNITPRICE,CURRENCY_RATE,WAERS,QTY
0,2100001642,00010,5000001438,00010,20/09/2021,A000,A01,000000000003000019,KALIUM CHLORIDE:KCL;FLAKE,591.00,14.23301,USD,5000.000
1,2200000473,00010,5100000823,00010,13/04/2016,B000,B01,000000000003000018,KALIUM CHLORIDE:KCL RED;K2O 60%MIN;,278.00,13.12300,USD,25000.000
2,2200000475,00010,5100000837,00010,14/04/2016,B000,B01,000000000003000018,KALIUM CHLORIDE:KCL RED;K2O 60%MIN;,278.00,13.23800,USD,25000.000
3,2200001384,00010,5100001812,00010,27/04/2016,B000,B01,000000000003000018,KALIUM CHLORIDE:KCL RED;K2O 60%MIN;,278.00,13.21500,USD,25000.000
4,2200001937,00010,5100002059,00010,04/05/2016,B000,B01,000000000001000017,DAP:(NH4)2HPO4;GRANULAR;18%N 46%P2O5,344.00,13.24600,USD,13500.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,2600033074,00010,5500027840,00010,03/12/2021,F000,F01,000000000003000018,KALIUM CHLORIDE:KCL;K2O 60%MIN,570.00,14.35301,USD,4800.010
433,2600033090,00010,5500027838,00010,03/12/2021,F000,F01,000000000003000018,KALIUM CHLORIDE:KCL;K2O 60%MIN,570.00,14.35301,USD,1831.937
434,2600033091,00010,5500027837,00010,03/12/2021,F000,F01,000000000003000018,KALIUM CHLORIDE:KCL;K2O 60%MIN,570.00,14.35301,USD,4700.500
435,2600033520,00010,5500028311,00010,25/01/2022,F000,F01,000000000003000018,KALIUM CHLORIDE:KCL;K2O 60%MIN,570.00,14.34701,USD,4850.380


In [275]:
df_dap = df_all[(df_all['EMATN'] == '000000000001000017') | (df_all['EMATN'] == '000000000003000499') | (df_all['EMATN'] == '000000000003000266') | (df_all['EMATN'] == '000000000003000539')]

df_dap = df_dap.loc[:,['BEDAT','UNITPRICE','WAERS']].sort_values(by='BEDAT')
df_dap.loc[df_dap["WAERS"] == "IDR", "UNITPRICE"] = df_dap[df_dap['WAERS'] == 'IDR']['UNITPRICE'].str[:-3]
df_dap['UNITPRICE'] = df_dap['UNITPRICE'].astype(float)
df_dap = pd.DataFrame(df_dap.loc[:,['BEDAT','UNITPRICE','WAERS']].groupby(['BEDAT','WAERS'])['UNITPRICE'].agg('median')).reset_index().sort_values('BEDAT')
df_dap

,BEDAT,WAERS,UNITPRICE
0,01/02/2018,IDR,52272.00
1,01/02/2018,USD,424.43
2,01/07/2020,USD,316.40
3,02/03/2020,USD,314.03
4,02/03/2022,USD,970.00
...,...,...,...
123,30/11/2017,USD,411.50
124,31/03/2021,USD,572.76
125,31/08/2017,USD,383.80
126,31/12/2017,IDR,69621.00


In [276]:
df_curr['Terakhir'] = df_curr['Terakhir'].str[:-2].str.replace('.','').astype(int)
df_curr = df_curr.loc[:,['Tanggal','Terakhir']]
df_curr

<ipython-input-276-a46f2bc0ae6a>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_curr['Terakhir'] = df_curr['Terakhir'].str[:-2].str.replace('.','').astype(int)


,Tanggal,Terakhir
0,09/05/2022,14557
1,28/04/2022,14497
2,27/04/2022,14420
3,26/04/2022,14410
4,25/04/2022,14455
...,...,...
1611,07/01/2016,13927
1612,06/01/2016,13947
1613,05/01/2016,13862
1614,04/01/2016,13915


In [277]:
for i in range(len(df_dap['BEDAT'])):
    for j in range(len(df_curr['Tanggal'])):
        if (df_dap.loc[i,'BEDAT'] == df_curr.loc[j,'Tanggal']) and df_dap.loc[i,'WAERS'] == 'IDR':
            idr_to_usd = df_dap.loc[i,'UNITPRICE']/df_curr.loc[j,'Terakhir']
            df_dap.loc[i, 'UNITPRICE'] = idr_to_usd
            df_dap.loc[i, 'WAERS'] = 'USD'
            
df_dap = df_dap.drop(df_dap[df_dap['WAERS']=='IDR'].index).reset_index(drop=True)
df_dap = df_dap.drop_duplicates(subset='BEDAT',keep='last').reset_index(drop=True)

df_dap['BEDAT'] = pd.to_datetime(df_dap['BEDAT'])
df_dap = df_dap.groupby(pd.Grouper(key='BEDAT', axis=0, 
                        freq='M')).mean().dropna().reset_index()

df_dap['BEDAT'] = df_dap['BEDAT'].astype(str).str[:4] + 'M' + df_dap['BEDAT'].astype(str).str[5:7]
df_dap

,BEDAT,UNITPRICE
0,2016M02,344.000000
1,2016M03,339.600000
2,2016M04,344.000000
3,2016M06,160.408778
4,2016M08,315.750000
5,2016M10,315.750000
6,2016M11,327.833333
7,2016M12,324.375000
8,2017M01,335.000000
9,2017M02,343.333333


# CEK KORELASI DENGAN DATA FERTECON

In [329]:
df_fertecon = pd.read_excel('Data Fertecon 2019 dst.xlsx',sheet_name=1)

In [330]:
df_fertecon.rename(columns={'date':'BEDAT'},inplace=True)

In [331]:
# df_fertecon.drop(columns='Unnamed: 15',inplace=True)
df_fertecon

,BEDAT,DAP/MAP fob US Gulf_min,DAP/MAP fob US Gulf_max,DAP fob N. Africa_min,DAP fob N. Africa_max,DAP fob China_min,DAP fob China_max,DAP cfr India_min,DAP cfr India_max,MAP fob Baltic_min,MAP fob Baltic_max,MAP cfr Brazil_min,MAP cfr Brazil_max,PhosAcid 100% P2O5 cfr India_min,PhosAcid 100% P2O5 cfr India_max
0,2019M01,405,417,415,450,391,405,405,417,400,415,425,435,750,768
1,2019M02,383,410,390,435,388,393,400,405,385,410,395,425,750,750
2,2019M03,373,391,390,430,382,391,397,402,365,400,395,410,750,750
3,2019M04,371,383,390,420,374,384,389,400,365,380,390,400,728,728
4,2019M05,352,379,355,415,355,371,370,388,352,373,370,390,728,728
5,2019M06,337,353,331,395,337,355,346,370,335,360,360,365,655,728
6,2019M07,320,342,331,372,334,340,346,352,317,337,345,357,655,655
7,2019M08,306,327,312,365,320,339,338,351,298,331,330,355,655,655
8,2019M09,292,326,308,340,311,323,329,339,286,305,320,330,655,655
9,2019M10,278,308,300,340,310,316,329,332,277,305,305,320,625,625


In [332]:
df_fertecon.merge(df_dap, on='BEDAT', how='left').dropna().corr()['UNITPRICE'].sort_values()

PhosAcid 100% P2O5 cfr India_max    0.600157
PhosAcid 100% P2O5 cfr India_min    0.600164
DAP cfr India_min                   0.604567
MAP fob Baltic_min                  0.626174
MAP fob Baltic_max                  0.626379
DAP/MAP fob US Gulf_max             0.631920
MAP cfr Brazil_max                  0.633681
DAP fob China_min                   0.641374
DAP fob N. Africa_max               0.642611
DAP/MAP fob US Gulf_min             0.643720
DAP cfr India_max                   0.645982
MAP cfr Brazil_min                  0.649110
DAP fob N. Africa_min               0.665876
DAP fob China_max                   0.670511
UNITPRICE                           1.000000
Name: UNITPRICE, dtype: float64